In [1]:
%reload_ext autoreload
%autoreload 2

In [19]:
from padding_oracle import CBCPaddingOracle
import requests
from copy import copy

def oracle(iv: bytes, block: bytes):
    # Create token with the iv and block
    token = iv + block

    # Create request and attach cookie with token
    cookie = {"authtoken": token.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)
    
    # Check response against some known error responses.
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or  r.content == b'Zero-length input cannot be unpadded'): #r.content == b'No quote for you!' or
        return False
    else:
        return True

# Establish baseline ciphertext
r = requests.get("http://localhost:5000")
ciphertext_str = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_str)

# Find block size by changing bytes from the iv end of the ciphertext until we get a padding error. 
# We know that the ciphertext contains an IV in the front.
ciphertext_bytes = copy(ciphertext_bytes_baseline)
block_size = 0
for i in range(len(ciphertext_bytes)):
    ciphertext_bytes[i] ^= 0x25
    r = requests.get("http://localhost:5000/quote", cookies={"authtoken": ciphertext_bytes.hex()})
    if (r.content != b'No quote for you!'):
        break
    else:
        block_size += 1

# Break encryption using the CBC Padding Oracle
breaker = CBCPaddingOracle(copy(ciphertext_bytes_baseline), block_size, oracle)
result = breaker.attack()

CBC-Padding-Oracle Attack
-------------------------
- IV + Ciphertext: 4aea74d0ae08ee00ffe6d2283b83a6027ca4f075bd07848490d6f0bab0d0b745d7bc99d89ae75301650d6c3e9070c8c061db0e4bd154099433213bfd1fe93271dad0eb6a3489b910accd3f15e0ec416de28922580dd57adc1d1561ec70b125ce17fe57a607b85affdd803ff44e0bfb32
- Block size: 16
- Block count: 7
-------------------------
Performing attack...
-------------------------
Block 0 / IV:      4aea74d0ae08ee00ffe6d2283b83a602
------------------------------------------------------------
Block 1:           039511e0d07d88759dd6a4514ce6c477                                                                                                                                                                         
DEC-Block 1:       138501f0c06d98658dc6b4415cf6d467
Plaintext-Block 1: 596f75206e6576657220666967757265
------------------------------------------------------------
Block 2:           4cdb95118d63fcf5f4e6c2e380b3cf3a                                               